This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [4]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [5]:
# First, import the relevant modules
import requests

In [8]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# Saving base url for querying
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?'
# Requesting data with limit parameter to get only one day (row)
r = requests.get(url + 'limit=1&' + 'api_key=' + API_KEY)
# Grabbing json from request object
r.json()

{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2019-02-05',
  'frequency': 'daily',
  'data': [['2019-02-05',
    None,
    82.6,
    80.5,
    82.05,
    None,
    None,
    None,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

Inspect the JSON structure of the object you created, and take note of how nested it is,
as well as the overall structure

The data printout comprises of a dict (json) for all data including metadata. The dataset metadata includes info for all available arguments to the API call and a list of the column names. The 'data' key is a list list nesting of all the actual data available for each row (indexed by date). Overall data structure doesn't seem too convoluted from the single row printout above.

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [9]:
# Using base url and start_date and end_date parameters to filter for just 2017 data
req_2017 = requests.get(url + 'start_date=2017-01-01&end_date=2017-12-31&' + 'api_key=' + API_KEY)

2. Convert the returned JSON object into a Python dictionary.

In [14]:
# Cast json into dict 
dict_2017 = dict(req_2017.json())

In [101]:
# dict_2017

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [31]:
# Extract all opening prices if available with list comp.
open_all = [e[1] for e in dict_2017['dataset_data']['data'] if e[1] is not None] 
# Get maximum val of opening prices
open_high = max(open_all)
open_high

53.11

In [34]:
# Get min val for opening prices
open_low = min(open_all)
open_low

34.0

4. What was the largest change in any one day (based on High and Low price)?

In [36]:
# Extract differences in the high and low prices of all days
diff_all = [e[2] - e[3] for e in dict_2017['dataset_data']['data']]
# Take max to find largest change
diff_max = max(diff_all)
diff_max

2.8100000000000023

5. What was the largest change between any two days (based on Closing Price)?

In [61]:
# Saving entire list of lists for working with indices
dat = dict_2017['dataset_data']['data']
# Calculate all change in closing price (start from 2nd latest day, take abosulute difference from previous element)
price_change_all = [abs(dat[i][4] - dat[i-1][4]) for i in range(1,len(dat))]

In [63]:
# Take max from price change list
max_change = max(price_change_all)

2.559999999999995

6. What was the average daily trading volume during this year?

In [71]:
# Extract traded volume for each day into list
volume_all = [e[6] for e in dict_2017['dataset_data']['data']]
# Calulate average
volume_avg = sum(volume_all) / len(dat)
volume_avg

89124.33725490196

7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [108]:
# Basic function for calculating median
def calc_median(val_list): # Takes in a list of values for which the median will be calculated
    val_list_sorted = sorted(val_list)    # sort given list
    val_count = len(val_list_sorted)      # Store len of list as the value count
    if val_count % 2 == 0:                # If count is even, take sum of 2 middle values
        med = sum([val_list_sorted[val_count // 2 - 1], val_list_sorted[val_count // 2]]) / 2
    else: med = val_list_sorted[val_count // 2]  # If count is odd, take middle position value
    return med

In [110]:
# Call calc_median on list of traded volumes extracted above
calc_median(volume_all)

76286.0